In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

M = 4
# Create 4 single-server queues with FCFS scheduling
node = np.empty(4, dtype=object)
node[0] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[1] = Queue(model, 'Queue2', SchedStrategy.FCFS)
node[2] = Queue(model, 'Queue3', SchedStrategy.FCFS)
node[3] = Queue(model, 'Queue4', SchedStrategy.FCFS)  # only closed classes

source = Source(model, 'Source')
sink = Sink(model, 'Sink')

# Create job classes
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 100, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

# Set service rates
for i in range(M):
    node[i].set_service(jobclass[0], Exp(i+1))
    node[i].set_service(jobclass[1], Exp(np.sqrt(i+1)))

# Set arrival rate for open class using APH distribution
source.set_arrival(jobclass[1], APH.fitMeanAndSCV(3, 64))

In [3]:
# Setup routing matrix
M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)
P.set(jobclass[0], jobclass[0], node[2], node[3], 1.0)
P.set(jobclass[0], jobclass[0], node[3], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Sink (excludes Queue4)
P.set(jobclass[1], jobclass[1], source, node[0], 1.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)
P.set(jobclass[1], jobclass[1], node[2], sink, 1.0)

model.link(P)

In [4]:
print('This example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.')# Create solverssolver = np.array([], dtype=object)# Note: CTMC is infinite on this model due to high population so omitted CTMC and SSAsolver = np.append(solver, SolverJMT(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=20000))#solver = np.append(solver, SolverFluid(model, keep=False, verbose=True, seed=23000)) #TODO Fluid is very slowsolver = np.append(solver, SolverMVA(model, method='lin', keep=False, verbose=True, seed=23000))solver = np.append(solver, SolverNC(model, keep=False, verbose=True, seed=23000))solver = np.append(solver, SolverMAM(model, keep=False, verbose=True, seed=23000))# Solve and display resultsAvgTable = np.empty(len(solver), dtype=object)for s in range(len(solver)):    print(f'\nSOLVER: {solver[s].get_name()}')    AvgTable[s] = solver[s].get_avg_table()

This example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.

SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/8354047143952879044/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 2.759755s.
  Station     JobClass     QLen    Util     RespT    ResidT    ArvR    Tput
0  Queue1  ClosedClass  98.4400  0.6812  143.7537  143.7537  0.6927  0.6887
1  Queue1    OpenClass  56.6240  0.2977  187.5381  187.5381  0.3042  0.3047
2  Queue2  ClosedClass   0.8707  0.3381    1.2547    1.2547  0.6887  0.6886
3  Queue2    OpenClass   0.5367  0.1911    1.7508    1.7508  0.3047  0.3047
4  Queue3  ClosedClass   0.4373  0.2063    0.6263    0.6263  0.6886  0.6927
5  Queue3    OpenClass   0.2972  0.1808    0.9696    0.9696  0.3047  0.2954
6  Queue4  ClosedClass   0.2189  0.1596    0.3142    0.3142  0.6927  0.6927
9  Source    OpenClass   0.0000  0.0000    0.0000    0.0000  0.0000  0.3042

SOLVER: SolverMVA
MVA analysis [method: lin, lang: java, env: 